In [53]:
import pandas as pd
from numpy import isnan
import vlc
import json
import urllib.request
import pickle

In [19]:
allbirds=pd.read_csv("Data/AvesEspaña.csv", header=None, sep=";")

# Get column names
column_names=["sp_latin", "ssp", "sp_ingles", "sp_espanol", "PB", "NA", "CA", "rareza"]
allbirds.columns=column_names

# Take family name and put it as new column
family=[]
lastfamily=None
for row in allbirds.iterrows():
    if not isinstance(row[1][3], str):
        lastfamily=row[1][0].lower()
    else:
        family.append(lastfamily)
        
# Remove family rows
allbirds=allbirds[allbirds["sp_ingles"].apply(lambda x : isinstance(x, str))]
allbirds["family"]=family

allbirds["sp_latin"]=allbirds["sp_latin"].apply(lambda x : x.lower())
allbirds.head()

,sp_latin,ssp,sp_ingles,sp_espanol,PB,NA,CA,rareza,family
1,branta bernicla,"bernicla, hrota",Brant Goose,Barnacla carinegra,A,NaN,A,R2 (solo ssp. hrota).,anatidae
2,branta ruficollis,NaN,Red-breasted Goose,Barnacla cuellirroja,A (i),NaN,NaN,R1,anatidae
3,branta leucopsis,NaN,Barnacle Goose,Barnacla cariblanca,A (i),NaN,D,NaN,anatidae
4,anser caerulescens,atlanticus,Snow Goose,Ánsar nival,A (i),NaN,NaN,R2,anatidae
5,anser anser,anser,Greylag Goose,Ánsar común,A (i),A,A(i),NaN,anatidae


In [26]:
def make_url(bird_sp):
    '''Given the scientific name of a bird species this function
       returns the url to retrieve the xeno-canto recordings '''
    
    query="https://www.xeno-canto.org/api/2/recordings?query="
    genus, name = bird_sp.split(" ")
    completeQuery=query+genus+"+"+name+"+q:A+cnt:Spain"
    return completeQuery

def get_recordings(url):
    '''Makes a request to retrieve a list of all the recordings 
       for the parameters specified in url'''
    
    with urllib.request.urlopen(url) as response:
        data = json.loads(response.read().decode())
        return data["recordings"]
    
bird="branta bernicla"
url=make_url(bird)
get_recordings(url)

[{'id': '631977',
  'gen': 'Branta',
  'sp': 'bernicla',
  'ssp': 'nigricans',
  'en': 'Brant Goose',
  'rec': 'Steve Hampton',
  'cnt': 'United States',
  'loc': 'Pt Hudson, Port Townsend, Jefferson County, Washington',
  'lat': '48.1176',
  'lng': '-122.7494',
  'alt': '0',
  'type': 'call',
  'url': '//www.xeno-canto.org/631977',
  'file': '//www.xeno-canto.org/631977/download',
  'file-name': 'XC631977-Brant Mar 25b.mp3',
  'sono': {'small': '//www.xeno-canto.org/sounds/uploaded/BUGKVZBWKF/ffts/XC631977-small.png',
   'med': '//www.xeno-canto.org/sounds/uploaded/BUGKVZBWKF/ffts/XC631977-med.png',
   'large': '//www.xeno-canto.org/sounds/uploaded/BUGKVZBWKF/ffts/XC631977-large.png',
   'full': '//www.xeno-canto.org/sounds/uploaded/BUGKVZBWKF/ffts/XC631977-full.png'},
  'lic': '//creativecommons.org/licenses/by-nc-sa/4.0/',
  'q': 'A',
  'length': '0:23',
  'time': '19:00',
  'date': '2021-03-25',
  'uploaded': '2021-03-26',
  'also': ['Larus glaucescens'],
  'rmk': 'group of 225 bir

In [55]:
# birdRecBySp={}
# for i, bird in enumerate(allbirds["sp_latin"]):
#     print(100*(i/len(allbirds)))
#     if len(bird.split(" "))==2:
#         birdRecBySp[bird]=get_recordings(make_url(bird))

with open('Data/birdRecBySpDict.pickle', 'wb') as handle:
    pickle.dump(birdRecBySp, handle, protocol=pickle.HIGHEST_PROTOCOL)

#with open('Data/birdRecBySpDict.pickle', 'rb') as handle:
#    birdRecBySp = pickle.load(handle)

In [51]:
url=birdRecBySp["troglodytes troglodytes"][8]["file"]
url

'//www.xeno-canto.org/643490/download'

In [49]:
includedBirds=list(birdRecBySp.keys())

['branta bernicla',
 'branta ruficollis',
 'branta leucopsis',
 'anser caerulescens',
 'anser anser',
 'anser fabalis',
 'anser brachyrhynchus',
 'anser albifrons',
 'cygnus olor',
 'cygnus columbianus',
 'cygnus cygnus',
 'tadorna tadorna',
 'tadorna ferruginea',
 'spatula querquedula',
 'spatula discors',
 'spatula clypeata',
 'mareca strepera',
 'mareca penelope',
 'mareca americana',
 'anas platyrhynchos',
 'anas rubripes',
 'anas acuta',
 'anas crecca',
 'anas carolinensis',
 'marmaronetta angustirostris',
 'netta rufina',
 'aythya ferina',
 'aythya nyroca',
 'aythya collaris',
 'aythya fuligula',
 'aythya marila',
 'aythya affinis',
 'somateria spectabilis',
 'somateria mollissima',
 'melanitta perspicillata',
 'melanitta fusca',
 'melanitta deglandi',
 'melanitta nigra',
 'melanitta americana',
 'clangula hyemalis',
 'bucephala albeola',
 'bucephala clangula',
 'bucephala islandica',
 'mergellus albellus',
 'mergus merganser',
 'mergus serrator',
 'oxyura leucocephala',
 'tetras

In [45]:
def question(url, bird=None, possible=None):
    p = vlc.MediaPlayer(url)
    p.play()
    print("Enter bird name:")
    res=bird
    ans=input()
    if ans == res:
        print("Bieeeen")
    else:
        print("maaaaal")
    p.set_pause(1)
#url="https://www.xeno-canto.org/sounds/uploaded/ZWAQHOJFLZ/XC477551-190503-Troglodyte%20mignon%40Sacharewo.mp3"
question("https:"+url, "Cargolet")

Enter bird name:
qwertw
maaaaal
